In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sklearn
!pip install h5py

In [ ]:
import pandas as pd
# Delimiter the word file
a = pd.read_csv('/content/drive/MyDrive/datadir/words_4.txt',names=["col1","col2","col3","col4","col5","col6","col7", "col8","col9","col10","col11"],low_memory=False, delimiter=" ")
# Printing the txt file length to see total words and cross check with the number of images
print(len(a))

# b = a[['col9', 'col10','col11']]
# print(b)

# Combining the rows of the columns using apply and lambda functions
a['combined'] = a[['col9', 'col10','col11']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Removing the NaN values from the combined columns
for i in range(len(a['combined'])):
    a['combined'][i] = a['combined'][i].replace("nan", "")
    a['combined'][i] = a['combined'][i].replace(" ", "")

a['combined'].head()

from sklearn.preprocessing import LabelEncoder
# Lable Encoding
le = LabelEncoder()
a['combined_2'] = le.fit_transform(a['combined'])
# getting unique value to the combine cols
uniqueValues = a['combined_2'].unique()
print(len(uniqueValues))
outputCount = len(uniqueValues)

In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Load the word file
a = pd.read_csv('/content/drive/MyDrive/datadir/words_4.txt', names=["col1","col2","col3","col4","col5","col6","col7", "col8","col9","col10","col11"], low_memory=False, delimiter=" ")

# Combine the rows of the columns
a['combined'] = a[['col9', 'col10','col11']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Remove the NaN values from the combined columns
a['combined'] = a['combined'].str.replace("nan", "").str.replace(" ", "")

# Label Encoding
le = LabelEncoder()
a['combined_2'] = le.fit_transform(a['combined'])
outputCount = len(le.classes_)

# Define image dimensions and batch size
img_height = 256
img_width = 256
batch_size = 18

# Data augmentation and image preprocessing
datagen = ImageDataGenerator(rescale=1.0/255.0)

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(128, (3,3), activation='relu', input_shape=(img_height,img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3,3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(outputCount))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

# Training loop
epochs = 100
train_dir = "/content/drive/MyDrive/imgs"
train_ds = datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=batch_size, class_mode='sparse')

history = model.fit(train_ds, epochs=epochs)

# Save the trained model
model.save('/content/drive/MyDrive/data_dir/model_history.h5')

# Load the trained model
model = models.load_model('/content/drive/MyDrive/data_dir/model_history.h5')

In [ ]:
# Load and preprocess the test image
test_image_path = "/content/test.png"
img = cv2.imread(test_image_path)
img = cv2.resize(img, (img_width, img_height))
img = img.reshape(1, img_height, img_width, 3)
img = datagen.standardize(img)

# Make predictions on the test image
prediction = model.predict(img)
predicted_class = np.argmax(prediction)
predicted_word = a["combined"][predicted_class]
print(f"Prediction: {predicted_word}")